In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import anndata

In [ ]:
# definindo os parametros globais de qualidade do SCANPY
sc.settings.verbosity = 2  # verbosity: erro (0) perigo (1), info (2), dicas (3)
sc.settings.set_figure_params(dpi=600, dpi_save=600, format ='svg',) #qualidade das imagens para 700 DPI (alta resolução)
sc.settings.n_jobs = 8 #numero de nucleos utilizado no processamento paralelo (+ core, mais rapido os calculos serão realizados)

In [ ]:
# lendo os dados de single cell no formato h5ad
adata = sc.read_h5ad('shalek_nasal.h5a')

In [ ]:
adata

# Análise exploratória dos dados

In [ ]:
#Verificando o tipo e o numero de células
ncell_1=adata.obs[['Annotation']].value_counts()
ncell_1
#ncell_1.to_excel("ncell_1.xlsx")

In [ ]:
#observado subgrupos de células
ncell_2=adata.obs[['Detailed_Cell_Annotations']].value_counts()
ncell_2
#ncell_2.to_excel("ncell_2.xlsx")

In [ ]:
# verificando a coorte e os numeros de individuos em cada tipo
coorte=adata.obs[['sex']].value_counts()
coorte
#coorte.to_excel("coorte.xlsx")

In [ ]:
#verificando o numero de individuos infectados
status=adata.obs[['SARSCoV2_PCR_Status']].value_counts()
status
#status.to_excel("status.xlsx")

In [ ]:
#Verificando o numero de pacientes em cada nivel sintmatico de clasificação WHO
groups=adata.obs[['Cohort_Disease_WHO_Score']].value_counts()
groups
#groups.to_excel("groups.xlsx")

# Pre-processamento dos dados

In [ ]:
#verificando os genes que geram maior fração de contagens em cada célula ou em todas as células
sc.pl.highest_expr_genes(adata, n_top=10, )


In [ ]:
# filtrando células que tenham no minimo 200 genes expressos
sc.pp.filter_cells(adata, min_genes=200)

In [ ]:
# filtrando genes que são detectados em menos de 3 células 
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata # visualizando os dados

verificando o número de variáveis.
antes tinhamos --> AnnData object with n_obs × n_vars = 32588 × 32871 (genes x Células).
agora temos ---> AnnData object with n_obs × n_vars = 31736 × 30214 (genes x células).
então filtramos um total de 852 genes, e 2657 células.

In [ ]:
# verificando a quantidade de células após a filtragem, pode acontecer de um cluster inteiro desaparecer, para este estudo.
filter_ncell=adata.obs[['Annotation']].value_counts()
filter_ncell
#filter_ncell.to_excel("filter_ncell.xlsx")

In [ ]:
#Células de baixa qualidade / morrendo frequentemente exibem contaminação mitocondrial extensa
#Células de baixa qualidade ou gotículas vazias geralmente têm poucos genes
#Dupletos ou multipletos de células podem exibir uma contagem de genes aberrantemente alta
# abaixo podemos verificar essas porcentagens
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # anotar o grupo de genes mitocondriais como 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
# visualizando as procentagens em grafico de violino, isso permite verificar a variação de contagens, para uma proxima filtragem
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

Em "numero de genes por contagens" observamos que estão com mais de 8000 contagens em relação a contagens totais, isso pode ser ocasionado por dupletos ou multipletos que amplifica a contagem de genes.
observamos também a contagem de genes mitocondriais, que esta maior que 10 em algumas células, estas podem ser removidos das porcentagens, porque provavelmente pode ser derivado de viés de bancada/ (células mortas)

In [ ]:
# verificar na escala para delimitar um corte para o proximo filtro
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
#Filtragem para tirar genes com altas contagens (baseado nas observações dos graficos logo acima)
adata = adata[adata.obs.n_genes_by_counts < 6000, :]
adata = adata[adata.obs.pct_counts_mt < 7, :]

In [ ]:
# verificando novamente as contagens baseadas no filtro acima, podemos obervar que funcionou como esperado.
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
# etapa demorada, descomentar e utilizar se necessario!
# verificando se as contagens estão normalizadas
# podemos extrais a matriz de contagens do anndata e verificar com cuidado em um arquivo .csv
#t=adata.raw.X.toarray()
#pd.DataFrame(data=t, index=adata.obs_names, columns=adata.raw.var_names).to_csv('adata_raw_x.csv')

In [ ]:
#Depois de remover células indesejadas do conjunto de dados, a próxima etapa é normalizar os dados
#Por padrão empregamos um método de normalização de escala global " normalize_total" que normaliza as medidas
#de expressão de recurosos para cada célula pela expressão totol, multiplica isso por um fator de (10.000 por padrão)
# ou pelo tamanho da biblioteca ( consultar grafico de violino acima para total counts) e depois
#tranformamos para log o resultado da normalização.

In [ ]:
# normalizando a contagem total para 10.000 de modo que as contagens se tornem compareveis entre as células
sc.pp.normalize_total(adata, target_sum=1e6)

In [ ]:
# logaritmizando os dados
sc.pp.log1p(adata)


In [ ]:
# identificação de genes altamente variaveis
# obs: para salvar a lista existe o consumo de muita memoria, 
#então criar somente o objeto dentro do andata pode ser a melhor opção para uso posterior


Em seguida, calculamos um subconjunto de recursos que exibem alta variação de célula para célula no conjunto de dados (ou seja, eles são altamente expressos em algumas células e pouco expressos em outras). (Heisler M. et al 2013) descobriram que focar nesses genes na análise downstream ajuda a destacar o sinal biológico em conjuntos de dados de uma única célula.
consultar ---> https://www.nature.com/articles/nmeth.2645

In [ ]:
#identificndo os genes altamente variaveis e anotando em um objeto anndata para consultas posteriores
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5 )


In [ ]:
#plotando os genes altamente variaveis e não altamente variaveis
sc.pl.highly_variable_genes(adata)


In [ ]:
adata # apenas para vaerificar a criaçãos dos objetos no anndata 


In [ ]:
# código para extrair a matriz de genes altamente variaveis (demora e consome muita memoria)
#v_gen=adata.raw.to_adata()
#pd.DataFrame(data=v_gen, index=adata.obs_names, columns=adata.var.highly_variable).to_csv('genes_altamente_variaveis.csv', sep =',')

In [ ]:
# Agora aplicamos uma trasnformação linear ("dimensionamento") que é uma etapa de pré-processamento padrão antes de esecutar
#as tecnicas de redução de dimencionalidade como PCA, UMAP,TSNE...
#(muda a expresão de cada gene, de modo que a expressao media nas células seja 0)
#(dimensiona a expressão de cada gene de modo que a variancia entre as células seja 1) --> esta etapa da peso igual nas analises Downstream, 
#de modo que genes altamente expressos não dominem o resultado.


In [ ]:
# Regressar os efeitos de contagem total por célula e a porcentagem de genes mitocondriais expresso.Dimencione os dados de acordo com a variação de unidade.
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
#Dimensione cada gene de acordo com a variância unitária. Os valores do clipe excedem o desvio padrão 10.
sc.pp.scale(adata, max_value=10)

# Expressão diferencial de Genes

Apartir daqui não iremos abordar a descoberta de novas células. Para a desberta de células um outro fluxo de trabalho deriva neste ponto, utilizando de genes marcadores de células e agrupamento n supervisionado. Após sua anotação de novos grupos de células vc poderá começar deste ponto novamente.
OBS: Ao verificar as anotações você poderá pensar em como realizar sua analise de genes diferencialmente expressos, podendo ser entre os grupos de células, condição da coorte, sexo, habitos...

In [ ]:
#Grupos a serem testados na expressão diferencial
coorte

In [ ]:
#realizando a analise de genes diferencialmente expressos entre os grupos
sc.tl.rank_genes_groups(adata, 'sex', groups=['male'], 
    reference='female', method='t-test',corr_method ='benjamini-hochberg', key_added = 'DEG_male_vs_female')

In [ ]:
# extraindo a tabela de genes diferencialmene expressos
tab_dif = sc.get.rank_genes_groups_df(adata, key = "DEG_pos_vs_neg",group= None) #tabela total de DEGs
tab_dif_2 = sc.get.rank_genes_groups_df(adata, key = "DEG_pos_vs_neg",group= None, pval_cutoff = 0.001 , log2fc_min = -5.0 , log2fc_max = 5.0) #cut of seleciona oas genes masi significativos dentro do grupo <0.001

In [ ]:
# mostrando os resultados
#tab_dif_2
tab_dif

In [ ]:
#escrevendo os resultados em uma tabela do excel
tab_dif.to_csv("lista_DEG_pos_vs_neg_total.csv", sep=',')
#tab_dif_2.to_excel("lista_DEG_male_vs_female_filtrada_.xlsx")
tab_dif_2.to_csv("lista_DEG_pos_vs_neg_filtrada.csv",sep=',')

In [ ]:
#criando um novo check point para salvar todos osbjetos criado e em seguida aprimorar os detalhes
adata.write_h5ad('dados_shalek_processados_com_DEGs.h5ad')

# Visualizando os dados em um grafico de vulcão

In [ ]:
# importanto a biblioteca de visualizão de graficos para dados biologicos
from bioinfokit import analys, visuz


In [ ]:
#montando o volcano plot
visuz.GeneExpression.volcano(df=tab_dif_2, lfc="logfoldchanges", pv="pvals",gstyle=1, sign_line=True,plotlegend=True, legendpos='upper right', 
    legendanchor=(1.46,1), lfc_thr=(0.5, 0.5), pv_thr=(0.001, 0.001), geneid="names",xlm=None, ylm=None,r=300,dotsize= 1.5,axylabel='FDR',color=('#0000CD','#A4A4A4','#FF3030'),valpha=0.5)